# Regresión Lineal Simple. Un ejemplo minimalista

### Importar las librerías relevantes

In [18]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para graficar en 3-D
import nbformat as nbf

### Generar datos al azar para entrenar al modelo

Se trabajará con dos variables de entrada, las x1 y x2 en los ejemplos vistos en clase. Se generan al azar a partir de una distribución uniforme.

Se creará una matriz con estas dos variables.  La matriz X del modelo lineal y = x * w + b

In [19]:
# Por facilidad, se declara una variable para indicar el tamaño del conjunto 
#      de datos de entrenamiento. Puede probarse con 100,000 o 1,000,000 pero hay 
#      que tener cuidado ya que con tantas observaciones puede que la máquina se quede!

observaciones = 1000

x1 = np.random.uniform(low = -10, high = 10, size = (observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

X = np.column_stack((x1,x2))

# Verificar la forma de la matriz 
# Debiera ser n x k, donde n es el número de observaciones, y k es el número de variables.

print (X.shape)

(1000, 2)


### Generar las metas a las que debemos apuntar

Para el modelo se usará la función f(x1, x2) = 2 * x1 - 3 * x2 + 5 + <ruido pequeño>.  El ruido es para hacerlo más realista.

Se utiliza la metodología de ML, y al finalizar se determinará si el algoritmo la ha aprendido.  

Al utilizar esta función, se espera que la red neuronal genere los pesos w1 = 2, w2 = -3 y el sesgo b = 5.  Si no se logra, es que algo se ha hecho mal. 

In [20]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

metas = 13 * x1 + 7 * x2 - 12 + ruido

# Verificar las dimensiones. Deben ser n x m, donde m es el número de variables de salida.
print (metas.shape)

(1000, 1)


### Graficar los datos a usar para el entrenamiento

La idea es ver que haya una fuerte tendencia que el modelo debe aprender a reproducir.


In [21]:
print(x1.shape)
print(x2.shape)
print(metas.shape)

(1000, 1)
(1000, 1)
(1000, 1)


In [22]:
x1N = x1.reshape(observaciones,)
x2N = x2.reshape(observaciones,)
metasN = metas.reshape(observaciones,)

fig = px.scatter_3d(x = x1N, y = x2N, z = metasN)

fig.update_layout(
    width = 500,
    height = 500,)

fig.show()

### Inicializar variables

Se inicializan los pesos y sesgos, al azar, dentro de un rango inicial pequeño.  Es posible "jugar" con este valor pero no es recomendable ya que el uso de rangos iniciales altos inhibe el aprendizaje por parte del algoritmo

Los pesos son de dimensiones k x m, donde k es el numero de variables de entrada y m es el número de variables de salida.  

Como solo hay una salida, el sesgo es de tamaño 1, y es un escalar

In [23]:
rango_inicial = 0.1     #  valor máximo para los pesos y sesgos iniciales

pesos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size = (2, 1))

sesgos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size = 1)

#Ver cómo fueron inicializados.
print (pesos)
print (sesgos)

[[0.04884971]
 [0.01123255]]
[0.06318448]


In [24]:
pesos.shape

(2, 1)

### Asignar la tasa de aprendizaje (Eta)

Se asigna una tasa de aprendizaje pequeña.  Para este ejemplo funciona bien 0.02.  Vale la pena "jugar" con este valor para ver los resultados de hacerlo.

In [25]:
eta = 0.0001

### Entrenar el modelo

Se utilizará un valor de 100 para iterar el modelo con el conjunto de datos de entrenamiento.  Ese valor funciona bastante bien con la tasa de aprendizaje de 0.02.  Cómo saber el número adecuado de iteraciones es algo que se verá en futuras sesiones, pero generalmente una tasa de aprendizaje baja requiere de más iteraciones que una más alta.  Sin embargo hay que tener en mente que una tasa de aprendizaje alta puede causar que la pérdida "Loss" diverja a infinito, en vez de converger a cero (0)

Puesto que esta es una regresión, se usará la función de pérdida L2-norm (dividido por 2, para ser consistente con la clase).  Es más, también se dividirá por el número de observaciones para obtener un promedio de pérdida por observación.  Se discutió en clase sobre la libertad de modificar esta función una vez no se pierda la característica de ser más baja para los resultados mejores, y vice versa.

Se mostrará la función de pérdida (loss) en cada iteración, para ver si está decreciendo como se desea.

Otro pequeño truco es escalar las deltas de la misma manera que se hizo con la función de pérdida.  De esta forma la tasa de aprendizaje es independiente del número de observaciones.  De nuevo esto no cambia el principio, solo hace más fácil la selección de una tasa única de aprendizaje. 

Finalmente se aplica la regla de actualización del decenso de gradiente.

Ojo!  los pesos son de dimensión 2 X 1, la tasa de aprendizaje es 1 X 1 (escalar), las entradas son 1000 X 2, y las deltas escaladas son 1000 X 1.  Es necesario obtener la transpuesta de las entradas para que no hayan problemas de dimensión en las operaciones. 



In [26]:
for i in range (100):
    
    # Esta es la ecuacion del modelo lineal: y = xw + b 
    y = np.dot(X, pesos) + sesgos
    
    # Las deltas son las diferencias entre las salidas y las metas (targets)
    # deltas es un vector 1000 x 1
    deltas = y - metas
        
    perdida = np.sum(np.abs(deltas)) / observaciones
    
    print(perdida)
    
    deltas_escaladas = deltas / observaciones
      
    pesos = pesos - eta * np.dot(X.T, deltas_escaladas)
    sesgos = sesgos - eta * np.sum(deltas_escaladas)

    #tiempo en que muestra el resultado
    
    # Los pesos son actualizados en forma de algebra lineal(una matriz menos otra)
    # Sin embargo, los sesgos en este caso son solo un número (solo se calcula una salida), 
    #       es necesario transformar las deltas a un escalar.      
    # Ambas líneas son consistentes con la metodología de decenso de gradiente

73.61947980854512
73.36376745525442
73.10895788265417
72.85508365964759
72.60222490652653
72.35025884186877
72.09918231252348
71.84899217647784
71.59968530281759
71.35125857168791
71.10371625724251
70.85712029961024
70.61139490206801
70.36655766498907
70.1226485053309
69.87961774576037
69.63750853716948
69.39625401953305
69.1558511739325
68.916296992113
68.6775884764458
68.43972263989065
68.20269650595853
67.96650710867424
67.73115149253928
67.49662671249497
67.2629298338854
67.03005793242077
66.79801814610235
66.56686964359776
66.33653710384621
66.10701764474521
65.87830839437264
65.65045176710863
65.423496214837
65.19737375924817
64.97204949438604
64.74752060093454
64.5237842695362
64.30083770075693
64.07867810505108
63.85730270272637
63.636760091875196
63.41702806670271
63.198071652050906
62.97988810836441
62.76251016556665
62.54594461548347
62.3301434792308
62.115104056816755
61.90082365778653
61.687299601188705
61.47452921554165
61.26250983880011
61.05123881832185
60.8407135108344

### Desplegar los pesos y el sesgo para ver si funcionaron correctamente.

Por el diseño de los datos, los pesos finales deben ser 2 y -3, y el sesgo: 5

**NOTA:**  Si aún no están los valores correctos, puede que aún estén convergiendo y sea necesario iterar más veces.  Para esto solo se requiere ejecutar la celda anterior cuantas veces sea requerido

In [27]:
print(pesos, sesgos)      

[[3.9004665 ]
 [2.09783243]] [-0.05268689]


### Graficar las últimas salidas vrs las metas (targets)

Como son los últimos valores, luego del entrenamiento, representan la exactitut del modelo final de.  Entre más cercana esté esta gráfica a una línea de 45 grados, más cercanas están las salidas y metas.

Como este ejemplo es pequeño, es posible hacerlo, en los problemas que se veran posteriormente en el curso, esto ya no sería posible.

In [28]:
yN = y.reshape(observaciones,)
metasN = metas.reshape(observaciones,)
fig = px.scatter(x = yN, y =  metasN)

fig.update_layout(
    width = 400,
    height = 400,)

fig.show()